<a href="https://colab.research.google.com/github/OneFineStarstuff/Cosmic-Brilliance/blob/main/Reverse_Causality_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
import tensorflow as tf

# Reproducibility
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

def make_reverse_causal_data(n_samples=12000, length=50, n_classes=10,
                             noise_std=0.12, burst_prob=0.10, seed=SEED):
    """
    Build labelable 1D time series:
    - Class c has its own frequency and phase.
    - A class-specific burst is more likely near the end (future), so backward context helps.
    Output: X (n, length, 1), y (n,)
    """
    rng = np.random.default_rng(seed)
    per_class = n_samples // n_classes
    X, y = [], []

    freqs  = np.linspace(0.6, 2.1, n_classes)
    phases = np.linspace(0.0, np.pi, n_classes)
    t = np.linspace(0, 1, length)
    last_win = slice(length - 10, length - 4)  # "future" window

    for c in range(n_classes):
        f, ph = freqs[c], phases[c]
        for _ in range(per_class):
            base = np.sin(2*np.pi*f*t + ph)
            trend = 0.3 * (t - 0.5) * ((c % 2) * 2 - 1)  # alternate up/down trend
            signal = 0.85*base + 0.15*trend

            # AR(1) noise for temporal correlation
            eps = rng.normal(0, noise_std, size=length)
            ar = np.zeros_like(eps)
            phi = 0.6
            for i in range(1, length):
                ar[i] = phi * ar[i-1] + eps[i]

            x = signal + ar

            # Future-coded burst (class-dependent amplitude)
            if rng.random() < burst_prob:
                amp = 0.8 + 0.15 * c  # class-specific intensity
                k = rng.integers(last_win.start, last_win.stop - 2)
                x[k:k+3] += rng.normal(amp, 0.05, size=3)

            X.append(x[:, None].astype("float32"))
            y.append(c)

    X = np.stack(X, axis=0)
    y = np.array(y, dtype=np.int32)

    # Shuffle
    idx = rng.permutation(len(X))
    return X[idx], y[idx]

# Data
X, y = make_reverse_causal_data(n_samples=12000, length=50, n_classes=10, noise_std=0.12, burst_prob=0.10)

# Split 70/15/15
n = len(X)
n_train = int(0.70 * n); n_val = int(0.15 * n)
X_train, y_train = X[:n_train], y[:n_train]
X_val,   y_val   = X[n_train:n_train+n_val], y[n_train:n_train+n_val]
X_test,  y_test  = X[n_train+n_val:],       y[n_train+n_val:]

# Standardize by train stats
mean = X_train.mean(axis=(0,1), keepdims=True)
std  = X_train.std(axis=(0,1), keepdims=True) + 1e-7
X_train = (X_train - mean) / std
X_val   = (X_val   - mean) / std
X_test  = (X_test  - mean) / std

# tf.data pipelines
BATCH = 128
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(8192, seed=SEED).batch(BATCH).prefetch(tf.data.AUTOTUNE)
val_ds   = tf.data.Dataset.from_tensor_slices((X_val, y_val)).batch(BATCH).prefetch(tf.data.AUTOTUNE)
test_ds  = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(BATCH).prefetch(tf.data.AUTOTUNE)

# Reverse Causal AI: forward (past) + backward (future) encoders with learned gate
class ReverseCausalAI(tf.keras.Model):
    def __init__(self, n_classes=10, width=64, l2=1e-5, drop=0.2):
        super().__init__()
        self.pre = tf.keras.layers.Conv1D(width, 5, padding='same', activation='relu',
                                          kernel_regularizer=tf.keras.regularizers.l2(l2))
        self.gru_f = tf.keras.layers.GRU(width, go_backwards=False,
                                         kernel_regularizer=tf.keras.regularizers.l2(l2))
        self.gru_b = tf.keras.layers.GRU(width, go_backwards=True,
                                         kernel_regularizer=tf.keras.regularizers.l2(l2))
        self.proj_f = tf.keras.layers.Dense(width, activation=None)
        self.proj_b = tf.keras.layers.Dense(width, activation=None)
        self.gate = tf.keras.layers.Dense(width, activation='sigmoid')
        self.dropout = tf.keras.layers.Dropout(drop)
        self.out = tf.keras.layers.Dense(n_classes, activation='softmax')

    def call(self, inputs, training=False):
        z = self.pre(inputs)                 # local temporal features
        h_f = self.gru_f(z)                  # past→present
        h_b = self.gru_b(z)                  # future→present (reverse-time summary)
        f = self.proj_f(h_f)
        b = self.proj_b(h_b)
        g = self.gate(tf.concat([f, b], axis=-1))  # learned reverse-causal gate
        fused = (1.0 - g) * f + g * b
        fused = self.dropout(fused, training=training)
        return self.out(fused)

model = ReverseCausalAI(n_classes=10, width=64, l2=1e-5, drop=0.2)

# Optimizer and compile
try:
    optimizer = tf.keras.optimizers.AdamW(learning_rate=1e-3, weight_decay=1e-4)
except AttributeError:
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Callbacks & logging
os.makedirs("results", exist_ok=True)
callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=8, restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3, min_lr=1e-5, verbose=1),
    tf.keras.callbacks.CSVLogger("results/training_history.csv"),
    tf.keras.callbacks.ModelCheckpoint("results/reverse_causal_best.keras",
                                       monitor="val_accuracy", save_best_only=True)
]

history = model.fit(train_ds, validation_data=val_ds, epochs=60, callbacks=callbacks, verbose=1)

# Evaluate
test_loss, test_acc = model.evaluate(test_ds, verbose=0)
print(f"✅ Test accuracy: {test_acc:.4f} | Test loss: {test_loss:.4f}")
print("✅ Training history saved to results/training_history.csv")
print("✅ Best model saved to results/reverse_causal_best.keras")

# Confusion matrix
y_pred = model.predict(test_ds, verbose=0).argmax(axis=1)
cm = tf.math.confusion_matrix(y_test, y_pred, num_classes=10).numpy()
print("Confusion matrix:\n", cm)